03 - Feature Engineering - Cancer Risk Prediction
=============================================
Yeni özellikler türeterek model performansını iyileştirme işlemleri ve yeni özellik(feature) değişkenleri oluşturularak modele dahil edilecektir. 

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv(
    "../data/raw/cancer-patient-data-sets.csv",
    encoding="latin1",
    engine="python"
)

df.shape

(1000, 26)

In [ ]:
base_features = [col for col in df.columns if col not in ['index', 'Patient Id', 'Level']]
print(f"Baseline Feature Count: {len(base_features)}")

# Kullanılmayan sütunlar silindi.

📝 Baseline Feature Count: 23


In [4]:
df_fe = df.copy() 

In [9]:
df_fe['Age_Group'] = pd.cut(
    df_fe['Age'], 
    bins=[0, 25, 40, 55, 100], 
    labels=[0, 1, 2, 3])  # Young, Adult, Middle, Senior olarak gruplandı.
df_fe['Age_Group'] = df_fe['Age_Group'].astype(int)



# Yaş(age) sayısal değişkeni kendi içinde gruplandırıldı.

In [24]:

df_fe['Environmental_Risk'] = (
    df_fe['Air Pollution'] + 
    df_fe['Dust Allergy'] + 
    df_fe['OccuPational Hazards'])

#  Risk Factor Score - Çevresel faktörler için bir grup oluşturuldu.

In [ ]:
df_fe['Lifestyle_Risk'] = (
    df_fe['Smoking'] + 
    df_fe['Alcohol use'] + 
    df_fe['Obesity'] +
    (10 - df_fe['Balanced Diet'])  # Tersi alındı çünkü daha yüksek diyet = daha düşük risk anlamına gelmektedir.
) / 4

# Yaşan tarzına(Lifestyle_Risk)  bağlı rick faktörleri için bir veri oluşturuldu.

In [11]:
df_fe['Genetic_Health_Risk'] = (
    df_fe['Genetic Risk'] + 
    df_fe['chronic Lung Disease']
) / 2

#  Genetik risk faktörüleri için bir veri hesaplandı.

In [22]:
symptom_cols = ['Chest Pain', 'Coughing of Blood', 'Fatigue', 'Weight Loss',
'Shortness of Breath', 'Wheezing', 'Swallowing Difficulty']
df_fe['Symptom_Severity'] = df_fe[symptom_cols].mean(axis=1)

#  Şiddetli belirtiler gruplandırıldı.

In [14]:
df_fe['Respiratory_Score'] = (
    df_fe['Shortness of Breath'] + 
    df_fe['Wheezing'] + 
    df_fe['Dry Cough'] +
    df_fe['chronic Lung Disease']
) / 4

# Solunum ile ilgili veriler gruplandı.

In [ ]:
critical_threshold = 6
df_fe['Critical_Symptom_Count'] = (
    (df_fe['Chest Pain'] >= critical_threshold).astype(int) +
    (df_fe['Coughing of Blood'] >= critical_threshold).astype(int) +
    (df_fe['Weight Loss'] >= critical_threshold).astype(int) +
    (df_fe['Shortness of Breath'] >= critical_threshold).astype(int) )

#  Kritik belirtiler alındı.

In [25]:
df_fe['Overall_Risk_Score'] = (
    df_fe['Environmental_Risk'] * 0.25 +
    df_fe['Lifestyle_Risk'] * 0.30 +
    df_fe['Genetic_Health_Risk'] * 0.20 +
    df_fe['Symptom_Severity'] * 0.25)

#  Tüm risk faktörleri alındı.

In [26]:
# Sigara ve yaş etkileşimi için bir değişken oluşturalım.
df_fe['Smoking_Age_Interaction'] = df_fe['Smoking'] * df_fe['Age']
print("✅ Smoking_Age_Interaction created")

# Genetik risk ve yaş arasındaki etkileşimi içeren bir değişken oluşturalım.
df_fe['Genetic_Age_Interaction'] = df_fe['Genetic Risk'] * df_fe['Age']
print("✅ Genetic_Age_Interaction created")

# Sigara ve hava kirliliği arasındaki etkileşimi içeren bir değişken oluşturalım.
df_fe['Smoking_Pollution'] = df_fe['Smoking'] * df_fe['Air Pollution']
print("✅ Smoking_Pollution created")

# Obezite ve kronik hastalık arasındaki etkileşimi kontrol edelim.
df_fe['Obesity_ChronicLung'] = df_fe['Obesity'] * df_fe['chronic Lung Disease']
print("✅ Obesity_ChronicLung created")

# Pasif sigara ve hava kirliliği arasındaki etkileşimi kontrol edelim.
df_fe['PassiveSmoker_Pollution'] = df_fe['Passive Smoker'] * df_fe['Air Pollution']
print("✅ PassiveSmoker_Pollution created")

✅ Smoking_Age_Interaction created
✅ Genetic_Age_Interaction created
✅ Smoking_Pollution created
✅ Obesity_ChronicLung created
✅ PassiveSmoker_Pollution created


In [ ]:
# Polinom özellikler(POLYNOMIAL FEATURES), mevcut bağımsız değişkenlerin (feature'ların) kuvvetlerini ($x^2, x^3$) ve bunların birbirleriyle olan çarpımlarını (etkileşim terimleri)
# veri setine ekler. Polinom özellikler, sadece bir özelliğin kendi kuvvetlerini değil, aynı zamanda iki veya daha fazla özelliğin etkileşimini de modellemeye olanak tanır.Böylece 
# birden fazla faktör için makine öğrenmesi modelinin doğrusal olmayan (non-linear) ilişkileri ve özellik etkileşimlerini yakalayabilmesini sağlamak amaçlanmaktadır.
# Belirli  değişkenler için polinom özellikler kontrol edildi.

important_features = ['Smoking', 'Air Pollution', 'Genetic Risk']

for feat in important_features:
    df_fe[f'{feat}_squared'] = df_fe[feat] ** 2
    print(f"✅ {feat}_squared created")

✅ Smoking_squared created
✅ Air Pollution_squared created
✅ Genetic Risk_squared created


In [29]:
# BINNING FEATURES (Özellikleri Gruplama/Kategorileştirme), sürekli (continuous) bir sayısal değişkenin değer aralığını belirli sayıda kategoriye veya "kutucuğa" (bin) bölme işlemidir.

df_fe['Smoking_Level'] = pd.cut(
    df_fe['Smoking'], 
    bins=[0, 2, 5, 10], 
    labels=[0, 1, 2])  # Low, Medium, High
df_fe['Smoking_Level'] = df_fe['Smoking_Level'].astype(int)
print("✅ Smoking_Level created")

# Hava kirliliği kategorileri
df_fe['Pollution_Level'] = pd.cut(
    df_fe['Air Pollution'], 
    bins=[0, 3, 6, 10], 
    labels=[0, 1, 2])
df_fe['Pollution_Level'] = df_fe['Pollution_Level'].astype(int)
print("✅ Pollution_Level created")

✅ Smoking_Level created
✅ Pollution_Level created


In [ ]:
new_features = [col for col in df_fe.columns if col not in df.columns and col != 'Level']
all_features = [col for col in df_fe.columns if col not in ['index', 'Patient Id', 'Level']]

print(f"\n✅ Original Features: {len(base_features)}")
print(f"✅ New Features: {len(new_features)}")
print(f"✅ Total Features: {len(all_features)}")

print(f"\n📝 New Feature List:")
for i, feat in enumerate(new_features, 1):
    print(f"   {i:2d}. {feat}")

# Oluşturulan veri setinin yeni hali hakkında özet.


✅ Original Features: 23
✅ New Features: 18
✅ Total Features: 41

📝 New Feature List:
    1. Age_Group
    2. Lifestyle_Risk
    3. Genetic_Health_Risk
    4. Symptom_Severity
    5. Respiratory_Score
    6. Critical_Symptom_Count
    7. Environmental_Risk
    8. Overall_Risk_Score
    9. Smoking_Age_Interaction
   10. Genetic_Age_Interaction
   11. Smoking_Pollution
   12. Obesity_ChronicLung
   13. PassiveSmoker_Pollution
   14. Smoking_squared
   15. Air Pollution_squared
   16. Genetic Risk_squared
   17. Smoking_Level
   18. Pollution_Level


In [32]:
# MODEL COMPARISON - Baseline vs Feature Engineered (MODEL KARŞILAŞTIRMASI: TEMEL VE ​​ÖZELLİK MÜHENDİSLİĞİ)

# Prepare data
X_baseline = df[base_features]
X_fe = df_fe[all_features]
y = df['Level']

# Train-test split
X_base_train, X_base_test, y_train, y_test = train_test_split(
    X_baseline, y, test_size=0.2, random_state=42, stratify=y
)
X_fe_train, X_fe_test, _, _ = train_test_split(
    X_fe, y, test_size=0.2, random_state=42, stratify=y
)

# Scaling
scaler_base = StandardScaler()
scaler_fe = StandardScaler()

X_base_train_scaled = scaler_base.fit_transform(X_base_train)
X_base_test_scaled = scaler_base.transform(X_base_test)

X_fe_train_scaled = scaler_fe.fit_transform(X_fe_train)
X_fe_test_scaled = scaler_fe.transform(X_fe_test)

# Train models
print("\n🔧 Training Baseline Model...")
model_baseline = LogisticRegression(random_state=42, max_iter=1000)
model_baseline.fit(X_base_train_scaled, y_train)

print("🔧 Training Feature Engineered Model...")
model_fe = LogisticRegression(random_state=42, max_iter=1000)
model_fe.fit(X_fe_train_scaled, y_train)

# Evaluate
baseline_train_acc = model_baseline.score(X_base_train_scaled, y_train)
baseline_test_acc = model_baseline.score(X_base_test_scaled, y_test)

fe_train_acc = model_fe.score(X_fe_train_scaled, y_train)
fe_test_acc = model_fe.score(X_fe_test_scaled, y_test)

# Cross-validation scores
print("\nPerforming Cross-Validation...")
cv_baseline = cross_val_score(model_baseline, X_base_train_scaled, y_train, cv=5)
cv_fe = cross_val_score(model_fe, X_fe_train_scaled, y_train, cv=5)


🔧 Training Baseline Model...
🔧 Training Feature Engineered Model...

Performing Cross-Validation...


In [33]:
print(f"""
BASELINE MODEL:
Train Accuracy:     {baseline_train_acc:.4f} ({baseline_train_acc*100:.2f}%)
Test Accuracy:      {baseline_test_acc:.4f} ({baseline_test_acc*100:.2f}%)
CV Score (mean):    {cv_baseline.mean():.4f} ± {cv_baseline.std():.4f}
Feature Count:      {len(base_features)}

FEATURE ENGINEERED MODEL:
Train Accuracy:     {fe_train_acc:.4f} ({fe_train_acc*100:.2f}%)
Test Accuracy:      {fe_test_acc:.4f} ({fe_test_acc*100:.2f}%)
CV Score (mean):    {cv_fe.mean():.4f} ± {cv_fe.std():.4f}
Feature Count:      {len(all_features)}

IMPROVEMENT:
Test Accuracy Gain: {(fe_test_acc - baseline_test_acc)*100:.2f}%
CV Score Gain:      {(cv_fe.mean() - cv_baseline.mean())*100:.2f}%
""")

if fe_test_acc > baseline_test_acc:
    print("✅ Feature Engineering başarılı! Model performansı arttı.")
else:
    print("⚠️ Feature Engineering beklenen etkiyi göstermedi. Revizyon gerekebilir.")

# Classification report
print("\n" + "="*80)
print("DETAILED CLASSIFICATION REPORT (Feature Engineered Model)")
print("="*80)
y_pred = model_fe.predict(X_fe_test_scaled)
print(classification_report(y_test, y_pred))


BASELINE MODEL:
Train Accuracy:     1.0000 (100.00%)
Test Accuracy:      1.0000 (100.00%)
CV Score (mean):    1.0000 ± 0.0000
Feature Count:      23

FEATURE ENGINEERED MODEL:
Train Accuracy:     1.0000 (100.00%)
Test Accuracy:      1.0000 (100.00%)
CV Score (mean):    1.0000 ± 0.0000
Feature Count:      41

IMPROVEMENT:
Test Accuracy Gain: 0.00%
CV Score Gain:      0.00%

⚠️ Feature Engineering beklenen etkiyi göstermedi. Revizyon gerekebilir.

DETAILED CLASSIFICATION REPORT (Feature Engineered Model)
              precision    recall  f1-score   support

        High       1.00      1.00      1.00        73
         Low       1.00      1.00      1.00        61
      Medium       1.00      1.00      1.00        66

    accuracy                           1.00       200
   macro avg       1.00      1.00      1.00       200
weighted avg       1.00      1.00      1.00       200



In [41]:

df_fe.to_csv('cancer_data_feature_engineered.csv', index=False)
print("✅ Feature engineered data saved: cancer_data_feature_engineered.csv")

# Save feature list
with open('feature_list.txt', 'w') as f:
    f.write("BASELINE FEATURES:\n")
    f.write("="*50 + "\n")
    for feat in base_features:
        f.write(f"- {feat}\n")
    
    f.write("\n\nNEW FEATURES:\n")
    f.write("="*50 + "\n")
    for feat in new_features:
        f.write(f"- {feat}\n")


✅ Feature engineered data saved: cancer_data_feature_engineered.csv
